In [1]:
import numpy as np
import copy
import json

In [2]:
root = '/notebooks/sinica/dataset/'

relation_data = root+'facial_r3.train'
schema_root = root+'schema_2.txt'
dev_data = root+'facial_r3.dev'
test_data = root+'skincare.dev'

UNKOWN_TAG = "<UNKNOWN>"
PAD_TAG = "<PAD>"
REL_NONE = 'Rel-None'
REL_PAD = 'Rel-Pad'

MAX_LEN = 80     # original 100

In [3]:
def readfile(data):
    with open(data, "r", encoding="utf-8") as f:
        content = f.read().splitlines()
        
    return content


def filter_MAX_LEN(content, MAX_LEN):
    init_loc = 0
    new_content = []
    
    for now_loc, now_token in enumerate(content):
        if now_token=='' and (now_loc-init_loc <= MAX_LEN):
            new_content.extend(content[init_loc:now_loc])
            new_content.append('')
            init_loc = now_loc+1
            
        elif now_token=='' and (now_loc-init_loc > MAX_LEN):
            init_loc = now_loc+1
        
    return new_content


# ================================================

def get_word_and_label(_content, start_w, end_w):
    word_list = []
    ent_list = []
    rel_list = []
    
    for word_set in _content[start_w:end_w]:
        word_set = word_set.split()
        if len(word_set)==1:
            word_list.append(' ')
            ent_list.append('O')
            rel_list.append(REL_NONE)
        
        else:
            word_list.append(word_set[0])
            ent_list.append(word_set[1])

            try:
                testerror = word_set[2]
            except IndexError:
                rel_list.append(REL_NONE)
            else:
                rel_list.append(word_set[2:])
    
    return word_list, ent_list, rel_list


def split_to_list(content):
    init = 0
    word_list = []
    ent_list = []
    rel_list = []

    for now_token, c in enumerate(content):
        if c=='':
            words, ents, rels = get_word_and_label(content, init, now_token)
            init = now_token+1
            word_list.append(words)
            ent_list.append(ents)
            rel_list.append(rels)
            
    return word_list, ent_list, rel_list

# ================================================


def gen_dataset_in_MAX_LEN(data, MAX_LEN):
    content = readfile(data)
    new_content = filter_MAX_LEN(content, MAX_LEN)
    
    return new_content

def save_file(file_name, dataset):      
    with open(file_name, 'w', encoding='utf8') as text_file:   
        for item in dataset:
            text_file.write("%s\n" % item)

In [4]:
new_train = gen_dataset_in_MAX_LEN(relation_data, MAX_LEN)
new_dev = gen_dataset_in_MAX_LEN(dev_data, MAX_LEN)
new_test = gen_dataset_in_MAX_LEN(test_data, MAX_LEN)

In [5]:
new_train_path = root+'facial_r3_len'+str(MAX_LEN)+'.train'
new_dev_path = root+'facial_r3_len'+str(MAX_LEN)+'.dev'
new_test_path = root+'facial_r3_len'+str(MAX_LEN)+'.test'

In [6]:
# word_list, ent_list, rel_list = split_to_list(new_dev)
# len(word_list), len(ent_list), len(rel_list)

In [7]:
save_file(new_train_path, new_train)
save_file(new_dev_path, new_dev)
save_file(new_test_path, new_test)